In [1]:
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import os
import numpy as np
from UNet import UNet

In [2]:
class SegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]  # 仅筛选出图像文件

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        
        # 获取图像路径和标签路径（假设标签文件以 .npy 结尾）
        img_path = os.path.join(self.image_dir, img_name)
        mask_name = img_name.rsplit('.')[0] + '.npy'  # 将图像文件后缀替换为 .npy
        mask_path = os.path.join(self.mask_dir, mask_name)

        # 加载灰度图像和标签
        image = Image.open(img_path).convert('L')  
        mask = np.load(mask_path)  # 加载标签

       # 图像预处理
        if self.transform:
            image = self.transform(image)  # 转换为 Tensor，并变为 [1, 200, 200]

        mask = torch.from_numpy(mask).long()  # 标签转换为 LongTensor，并保持形状为 [200, 200]
       


        return image, mask

# 设置图像和标签路径
image_dir = 'images\\test'
mask_dir = 'test_ground_truths'

# 定义图像预处理
transform = transforms.Compose([
    transforms.Resize((200, 200)),  # 确保图像大小为200x200
    transforms.ToTensor(),  # 转换为 [1, 200, 200] 的张量
])

# 创建数据集和数据加载器
test_dataset = SegmentationDataset(image_dir, mask_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [3]:
def compute_iou(preds, labels, class_ids):
    """
    计算指定类别的IoU。
    
    参数:
    preds (Tensor): 预测类别，形状为 [batch_size, height, width]
    labels (Tensor): 真实标签，形状为 [batch_size, height, width]
    class_ids (list): 要计算IoU的类别列表
    
    返回:
    ious (dict): 每个指定类别的IoU值
    """
    ious = {}
    for cls in class_ids:
        pred_cls = (preds == cls)  # 预测类别为cls的位置
        label_cls = (labels == cls)  # 真实标签为cls的位置
        
        intersection = (pred_cls & label_cls).sum().item()  # 交集
        union = (pred_cls | label_cls).sum().item()  # 并集
        
        if union == 0:
            ious[cls] = float('nan')  # 如果并集为0，IoU未定义
        else:
            iou = intersection / union
            ious[cls] = iou
    
    return ious

In [4]:

# 定义模型结构（确保模型结构与保存时一致）
model = UNet(in_channels=1, num_classes=4)

# 将模型加载到设备（CPU 或 GPU）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 加载保存的模型参数
model.load_state_dict(torch.load('model.pth', map_location=device))

class_ids = [1, 2, 3]  # 需要计算的类别

In [5]:
# 打开一个文本文件用于写入
with open('iou_results.txt', 'w') as f:
    with torch.no_grad():
        num = 1  # 初始化图像编号
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            # 前向传播得到输出
            outputs = model(images)  # 输出为 [batch_size, num_classes, height, width]

            # 预测类别，使用 argmax 得到每个像素点的类别预测
            preds = torch.argmax(outputs, dim=1)  # 预测类别 [batch_size, height, width]

            # 计算标签为1, 2, 3的 IoU
            ious = compute_iou(preds, labels, class_ids)
            # 将 IoU 结果写入文件
            f.write(f"IoUs for image {num}: {ious}\n")
            num += 1  # 增加图像编号
